# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Calculate to total number of schools
schools_unique = len(school_data_complete["school_name"].unique())

#Calculate the total number of students
students_unique = len(school_data_complete["Student ID"].unique())

# Using drop duplicates to get total budget and remove duplicates
Budget_data_clean = school_data_complete["budget"].drop_duplicates()

#Calculate the total budget
Budget_Total = Budget_data_clean.sum()

# Calculate the average math score
Avg_Math = school_data_complete["math_score"].mean()

# Calculate the average reading score
Avg_Reading = school_data_complete["reading_score"].mean()

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
Overall_Passing_Rate = (Avg_Math + Avg_Reading)/2

# Calculate the percentage of students with a passing math score (70 or greater)
Math_Passing_df = school_data_complete.loc[school_data_complete["math_score"]>=70]
Math_Passing = (Math_Passing_df["Student ID"].nunique()/students_unique)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
Reading_Passing_df = school_data_complete.loc[school_data_complete["reading_score"]>=70]
Reading_Passing = (Reading_Passing_df["Student ID"].nunique()/students_unique)*100  

# Create a dataframe to hold the above results
summary_df = pd.DataFrame({"Total Schools": [schools_unique],
                           "Total Students": [students_unique],
                           "Total Budget": [Budget_Total],
                           "Average Math Score": [Avg_Math],
                            "Average Reading Score": [Avg_Reading],   
                                "% Passing Math": [Math_Passing],
                           "% Passing Reading": [Reading_Passing], "% Overall Passing Rate": [Overall_Passing_Rate]})


# Optional: give the displayed data cleaner formatting
# Use Map to format all the columns
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,}".format)

                                       
summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [73]:
# Using GroupBy in order to separate the data into fields according to "school" values
#add type to of school
school_type = school_data_complete.groupby('school_name').first()["type"]

#take the first budget number for each school and disregard duplicates
School_Budget_series = school_data_complete.groupby('school_name').first()["budget"]

#calaculate the number of students in each school
School_Heads_series = school_data_complete.groupby('school_name').count()["Student ID"]

#calculate the total budget per student
Per_Student_Budget_series = (School_Budget_series / School_Heads_series)

#calculate average reading and math scores
Read_avg_series = school_data_complete.groupby('school_name').mean()["reading_score"]
Math_avg_series = school_data_complete.groupby('school_name').mean()["math_score"]

# Calculate the percentage of students with a passing math score (70 or greater)
Passing_Math_Perc_series = (school_data_complete[school_data_complete['math_score'] >=70].
                            groupby('school_name')['math_score'].count()/School_Heads_series)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
Passing_Read_Perc_series = (school_data_complete[school_data_complete['reading_score'] >=70].
                            groupby('school_name')['reading_score'].count()/School_Heads_series)*100

#calculate overall pass rate
Overall_Passing_Rate_series = (Passing_Math_Perc_series + Passing_Read_Perc_series)/2


#add values to dataframe
School_Summary_df = pd.DataFrame({
                        "School Type": school_type,
                           "Total Students": School_Heads_series,
                           "Total School Budget": School_Budget_series,
                            "Per Student Budget": Per_Student_Budget_series,
                           "Average Math Score": Math_avg_series,
                            "Average Reading Score": Read_avg_series,   
                                "% Passing Math": Passing_Math_Perc_series,
                           "% Passing Reading": Passing_Read_Perc_series, 
                            "% Overall Passing Rate": Overall_Passing_Rate_series})

# Use Map to format all the columns
School_Summary_df["Total Students"] = School_Summary_df["Total Students"].map("{:,}".format)
School_Summary_df["Total School Budget"] = School_Summary_df["Total School Budget"].map("${:,.2f}".format)                                       
School_Summary_df["Per Student Budget"] = School_Summary_df["Per Student Budget"].map("${:,.2f}".format)                                       

School_Summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
#sort schools in decending value for % Overall Passing Rate and filter to show top 5 schools
School_Summary_df.sort_values(by='% Overall Passing Rate', ascending=False).head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
#sort schools in ascending value for % Overall Passing Rate and filter to show bottom 5 schools
School_Summary_df.sort_values(by='% Overall Passing Rate', ascending=True).head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

Create a pandas series for each grade. Hint: use a conditional statement.

Group each series by school

Combine the series into a dataframe

Optional: give the displayed data cleaner formatting



In [7]:
#create groupings by school for each grade for average math score

######9th Grade############## 
Ninth_grade_math = school_data_complete[school_data_complete.grade == '9th'] 
Ninth_math_series = Ninth_grade_math.groupby('school_name').mean()["math_score"]

######10th Grade############## 
Tenth_grade_math = school_data_complete[school_data_complete.grade == '10th'] 
Tenth_math_series = Tenth_grade_math.groupby('school_name').mean()["math_score"]

######11th Grade############## 
Eleventh_grade_math = school_data_complete[school_data_complete.grade == '11th'] 
Eleventh_math_series = Eleventh_grade_math.groupby('school_name').mean()["math_score"]

######12th Grade############## 
Twelfth_grade_math = school_data_complete[school_data_complete.grade == '12th'] 
Twelfth_math_series = Twelfth_grade_math.groupby('school_name').mean()["math_score"]

#add values to dataframe 
Math_Grade_Summary_df = pd.DataFrame({ "9th": Ninth_math_series, 
                                      "10th": Tenth_math_series, 
                                      "11th": Eleventh_math_series, 
                                      "12th": Twelfth_math_series})

# Use Map to format columns
Math_Grade_Summary_df["9th"] = Math_Grade_Summary_df["9th"].map("{:.2f}%".format)
Math_Grade_Summary_df["10th"] = Math_Grade_Summary_df["10th"].map("{:.2f}%".format)
Math_Grade_Summary_df["11th"] = Math_Grade_Summary_df["11th"].map("{:.2f}%".format)
Math_Grade_Summary_df["12th"] = Math_Grade_Summary_df["12th"].map("{:.2f}%".format)
                                       

Math_Grade_Summary_df                                      



,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
#create groupings by school for each grade for average reading score

######9th Grade############## 
Ninth_grade_read = school_data_complete[school_data_complete.grade == '9th'] 
Ninth_read_series = Ninth_grade_read.groupby('school_name').mean()["reading_score"]

######10th Grade############## 
Tenth_grade_read = school_data_complete[school_data_complete.grade == '10th'] 
Tenth_read_series = Tenth_grade_read.groupby('school_name').mean()["reading_score"]

######11th Grade############## 
Eleventh_grade_read = school_data_complete[school_data_complete.grade == '11th'] 
Eleventh_read_series = Eleventh_grade_read.groupby('school_name').mean()["reading_score"]

######12th Grade############## 
Twelfth_grade_read = school_data_complete[school_data_complete.grade == '12th'] 
Twelfth_read_series = Twelfth_grade_read.groupby('school_name').mean()["reading_score"]

#add values to dataframe 
Read_Grade_Summary_df = pd.DataFrame({ "9th": Ninth_read_series, 
                                      "10th": Tenth_read_series, 
                                      "11th": Eleventh_read_series, 
                                      "12th": Twelfth_read_series})

# Use Map to format columns
Read_Grade_Summary_df["9th"] = Read_Grade_Summary_df["9th"].map("{:.2f}%".format)
Read_Grade_Summary_df["10th"] = Read_Grade_Summary_df["10th"].map("{:.2f}%".format)
Read_Grade_Summary_df["11th"] = Read_Grade_Summary_df["11th"].map("{:.2f}%".format)
Read_Grade_Summary_df["12th"] = Read_Grade_Summary_df["12th"].map("{:.2f}%".format)
                                       

     
Read_Grade_Summary_df                                      


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [96]:
#Create a table that breaks down school performances based on average Spending Ranges (Per Student). 
#add type to of school
# school_type = school_data_complete.groupby('school_name').first()["type"]

#take the first budget number for each school and disregard duplicates
# School_Budget_series2
# #calaculate the number of students in each school
# School_Heads_series = school_data_complete.groupby('school_name').count()["Student ID"]

# #calculate the total budget per student
# Per_Student_Budget_series = (School_Budget_series / School_Heads_series)

# #calculate average reading and math scores
Read_avg_series = School_Summary_df.groupby('Per_Student_Budget_series').mean()["reading_score"]
Read_avg_series
# Math_avg_series = school_data_complete.groupby('school_name').mean()["math_score"]

# # Calculate the percentage of students with a passing math score (70 or greater)
# Passing_Math_Perc_series = (school_data_complete[school_data_complete['math_score'] >=70].
#                             groupby('school_name')['math_score'].count()/School_Heads_series)*100

# # Calculate the percentage of students with a passing reading score (70 or greater)
# Passing_Read_Perc_series = (school_data_complete[school_data_complete['reading_score'] >=70].
#                             groupby('school_name')['reading_score'].count()/School_Heads_series)*100

# #calculate overall pass rate
# Overall_Passing_Rate_series = (Passing_Math_Perc_series + Passing_Read_Perc_series)/2

# #add values to dataframe
# School_Scores_Spend_df = pd.DataFrame({
#                         "School Type": school_type,
#                            "Total Students": School_Heads_series,
#                            "Total School Budget": School_Budget_series,
#                             "Per Student Budget": Per_Student_Budget_series,
#                            "Average Math Score": Math_avg_series,
#                             "Average Reading Score": Read_avg_series,   
#                                 "% Passing Math": Passing_Math_Perc_series,
#                            "% Passing Reading": Passing_Read_Perc_series, 
#                             "% Overall Passing Rate": Overall_Passing_Rate_series})

# # # Use Map to format all the columns
# # School_Scores_Spend_df["Total Students"] = School_Scores_Spend_df["Total Students"].map("{:,}".format)
# # School_Scores_Spend_df["Total School Budget"] = School_Scores_Spend_df["Total School Budget"].map("${:,.2f}".format)                                       
# # School_Scores_Spend_df["Per Student Budget"] = School_Scores_Spend_df["Per Student Budget"].map("${:,.2f}".format)                                       

# School_Scores_Spend_df




KeyError: 'Per_Student_Budget_series'

In [90]:
# Slice the data and place it into bins
# pd.cut(ted_df["views"], bins, labels=group_labels).head()
School_Scores_Spend_df["Spend Group"] = pd.cut(School_Scores_Spend_df['Spend Per Student'], spending_bins, labels=group_names)
School_Scores_Spend_df.head()


KeyError: 'Spend Per Student'

In [71]:
# Create a GroupBy object based upon "Spend per student Group"
student_spend_group = School_Scores_Spend_df.groupby("Spend Group")
# Get the average of each column within the GroupBy object
student_spend_group[["Average Math Score", "Average Reading Score", "% Passing Math", 
                     "% Passing Reading", "% Overall Passing Rate"]].sum()




# student_spend_group.fillna(0)
# student_spend_group
# # Find how many rows fall into each bin
# print(student_spend_group["Spend Group"].count())


,% Passing Math,% Passing Reading,% Overall Passing Rate
Spend Group,,,
<$585,0.000000,0.000000,0.000000
$585-615,0.000000,0.000000,0.000000
$615-645,74.980853,85.805463,80.431606
$645-675,0.000000,0.000000,0.000000


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Create a dataframe to hold the above results
School_Size_df = pd.DataFrame({ "School Name": [school_name],
                               "Number of Students": [students_count],
                            "Spend Per Student": [Per_Student_Budget],
                            "Average Math Score": [Avg_Math],
                            "Average Reading Score": [Avg_Reading],   
                                "% Passing Math": [Math_Passing],
                           "% Passing Reading": [Reading_Passing], "% Overall Passing Rate": [Overall_Passing_Rate]})



In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Slice the data and place it into bins
# pd.cut(ted_df["views"], bins, labels=group_labels).head()
School_Size_df["School Size Group"] = pd.cut(School_Size_df['School Name','Number of Students'], size_bins, labels=group_names)
School_Size_df.head()


## Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [District, Charter]
group_names = ["District", "Charter"]